## **Ingesta del archivo `races.csv`**

In [ ]:
# Creamos un parametro para el nombre del archivo
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
# Creamos un parametro para la fecha del archivo
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
# Llamamos al notebook que contiene las variables de configuración
%run "../utils/configuration"

In [ ]:
# Llamamos al notebook que contiene funciones comunes
%run "../utils/common_functions"

### Paso 1 - Crear tabla **f1_raw.races**

In [ ]:
%sql
DROP TABLE IF EXISTS f1_raw.races;
CREATE EXTERNAL TABLE IF NOT EXISTS f1_raw.races(raceId INT,
year INT,
round INT,
circuitId INT,
name STRING,
date DATE,
time STRING,
url STRING)
USING CSV
OPTIONS (path "/mnt/formula1dl/raw/races.csv", header true)

### Paso 2 - Leer el archivo CSV

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [ ]:
races_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                  StructField("year", IntegerType(), True),
                                  StructField("round", IntegerType(), True),
                                  StructField("circuitId", IntegerType(), True),
                                  StructField("name", StringType(), True),
                                  StructField("date", DateType(), True),
                                  StructField("time", StringType(), True),
                                  StructField("url", StringType(), True) 
])

In [ ]:
# El parámetro "raw_folder_path" se encuentra en el notebook "configuration"
# El parámetro "v_file_date" se encuentra en el notebook e indicamos su valor en tiempo de ejecución
races_df = spark.read \
.option("header", True) \
.schema(races_schema) \
.csv(f"{raw_folder_path}/races.csv")
#.csv(f"{raw_folder_path}/{v_file_date}/races.csv")

### Paso 3 - Añadir las columnas "ingestion_date" y "race_timestamp"

In [ ]:
from pyspark.sql.functions import to_timestamp, concat, col, lit

In [ ]:
races_with_timestamp_df = races_df.withColumn("race_timestamp", to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss')) \
                                  .withColumn("data_source", lit(v_data_source)) \
                                  .withColumn("file_date", lit(v_file_date))

In [ ]:
# La función "add_ingestion_date()" se encuentra en el notebook "common_functions"
# La fecha de ingestión será del tipo timestamp
races_with_ingestion_date_df = add_ingestion_date(races_with_timestamp_df)

### Paso 4 - Seleccionar sólo las columnas necesarias y renombrarlas como corresponda

In [ ]:
races_selected_df = races_with_ingestion_date_df.select(col('raceId').alias('race_id'), col('year').alias('race_year'), 
                                                        col('round'), col('circuitId').alias('circuit_id'),col('name'), col('ingestion_date'), col('race_timestamp'))

### Paso 5 - Escribir datos en el datalake como parquet y crear la tabla **races** en la base de datos **f1_processed**

In [ ]:
# Recordar que Databricks utiliza Hive para la gestión de tablas
# Esto quiere decir que una tabla en Hive se asocia con un directorio específico que contiene archivos de datos
# Aunque los datos subyacentes son solo archivos en un sistema de archivos, Hive proporciona una interfaz que presenta 
# estos datos como tablas.
# La base de datos tambien es un directorio. Podemos crearla manualmente, como con lenguaje SQL.
# Hive permite a los usuarios interactuar con esos datos utilizando un lenguaje similar a SQL
# En resumen, solo se esta creando un directorio y un archivo
# Escribimos el archivo con formato PARQUET en la base de datos "f1_processed" y en la tabla "races"
races_selected_df.write.mode("overwrite").partitionBy('race_year').format("parquet").saveAsTable("f1_processed.races")

In [ ]:
dbutils.notebook.exit("Success")